In [1]:
#run this for every file in the folder, so make a loop for that (execute this as a function)
file = 'filtered-bosnia-herzegovina-latest.osm.pbf'

In [2]:
from pipeline.buildings import create_geodataframe
gdf = create_geodataframe(file)

Loading osm-data from file filtered-bosnia-herzegovina-latest.osm.pbf, this will take a while...
invalid area (area_id=2330470704)
Creating geodataframe from the data from filtered-bosnia-herzegovina-latest.osm.pbf, this will take a while...


In [3]:
from pipeline.level_estimation import estimate_levels_with_knn
gdf = estimate_levels_with_knn(gdf)

Estimating levels for country bosnia-herzegovina, this will take a while...


In [4]:
from pipeline.climate_zones import get_climate_zones
zones = get_climate_zones()

In [5]:
from pipeline.energy_demand import get_energy_demand
gdf = get_energy_demand(gdf, zones)

Estimating energy demand per climate zone for country bosnia-herzegovina, this will take a while...


In [6]:
from pipeline.co2_emissions import get_co2_emissions
import pandas as pd
co2_rates = pd.read_csv('pipeline/co2rates.csv', index_col=0)
gdf = get_co2_emissions(gdf, co2_rates)

Calculating co2-emissions for country bosnia-herzegovina...


In [7]:
from pipeline.square_tiles import create_square_tiles
tuple = create_square_tiles(gdf)

Creating square-tiles of sum emission levels for bosnia-herzegovina, this will take a while...


In [8]:
from dotenv import dotenv_values
cf = dotenv_values(".env")
db_url = f'postgresql://{cf["USER"]}:{cf["PW"]}@{cf["URL"]}:{cf["PORT"]}/{cf["DB"]}'

In [9]:
from sqlalchemy import create_engine
engine = create_engine(db_url)  

In [10]:
gdf.to_postgis(cf["TABLE"], engine, if_exists="append") 